In [ ]:
from pathlib import Path
INPUT_DIR   = Path(r"working file")   
OUTPUT_ROOT = Path(r"output folder")  
WINDOW_SEC  = 6.0 # length of each small slice
RANDOM_SEED = 1234 # random seed for devide into train/val/test

In [ ]:
import math, random
from dataclasses import dataclass
from typing import List, Optional, Tuple

import numpy as np
import pandas as pd

In [ ]:
def _detect_time_col(df: pd.DataFrame) -> str:
    candidates = ["t_sec", "time", "Time", "t", "seconds", "sec"]
    for c in candidates:
        if c in df.columns:
            return c
    return df.columns[0]  

def _find_first_col_containing(df: pd.DataFrame, needle: str) -> Optional[str]:
    needle_lower = needle.lower()
    for col in df.columns:
        if needle_lower in str(col).lower():
            return col
    return None

def _continuous_runs(t: np.ndarray) -> List[Tuple[int, int]]:
    """
    return the time period
    """
    if t.size == 0:
        return []
    breaks = np.where(np.diff(t) < 0)[0]
    starts = [0] + (breaks + 1).tolist()
    ends   = (breaks + 1).tolist() + [t.size]
    return list(zip(starts, ends))

def _tolerance_from_dt(t: np.ndarray) -> float:
    """
    test the time length
    """
    if t.size < 2:
        return 0.01
    dt = np.median(np.diff(t))
    if not np.isfinite(dt) or dt <= 0:
        return 0.01
    return float(dt) * 1.5

In [ ]:
# split data into small part
def split_one_csv(csv_path: Path, window_sec: float = WINDOW_SEC):
    """
    return (segments, spans)
    - segments: List[pd.DataFrame] as [t_sec, ECG, PPG, ABP]
    - spans:    List[(start_time, end_time)]
    """
    try:
        df = pd.read_csv(csv_path)
    except Exception as e:
        print(f"[SKIP] read fail {csv_path.name}: {e}")
        return [], []

    if df.empty:
        print(f"[SKIP] empty file {csv_path.name}")
        return [], []

    # col name
    time_col = _detect_time_col(df)
    ecg_col  = _find_first_col_containing(df, "ECG")
    ppg_col  = _find_first_col_containing(df, "PPG")
    abp_col  = _find_first_col_containing(df, "ABP")

    missing = [n for n, c in [("time", time_col), ("ECG", ecg_col), ("PPG", ppg_col), ("ABP", abp_col)] if c is None]
    if missing:
        print(f"[SKIP] {csv_path.name}: no column: {', '.join(missing)}")
        return [], []
        
    for c in [time_col, ecg_col, ppg_col, abp_col]:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    df = df.dropna(subset=[time_col]).sort_values(time_col).reset_index(drop=True)

    t = df[time_col].to_numpy()
    runs = _continuous_runs(t)

    segments, spans = [], []
    for s, e in runs:
        run = df.iloc[s:e].copy()
        if len(run) < 2:
            continue
        t_run = run[time_col].to_numpy()
        tol   = _tolerance_from_dt(t_run)

        t0, t_last = float(t_run[0]), float(t_run[-1])
        total_dur  = t_last - t0
        if total_dur < window_sec - tol:
            # if very short
            continue

        # total files
        n_windows = int(math.floor((t_last - t0) / window_sec))

        for k in range(n_windows):
            w_start = t0 + k * window_sec
            w_end   = w_start + window_sec
            mask = (run[time_col] >= w_start) & (run[time_col] < w_end)
            seg  = run.loc[mask].copy()
            if seg.empty:
                continue

            # covered time
            cov = float(seg[time_col].iloc[-1] - seg[time_col].iloc[0])
            if cov < (window_sec - tol):
                continue

            # delete empty value
            if seg[[ecg_col, ppg_col, abp_col]].isna().any().any():
                continue

            # rename col name
            seg = seg[[time_col, ecg_col, ppg_col, abp_col]].rename(
                columns={time_col: "t_sec", ecg_col: "ECG", ppg_col: "PPG", abp_col: "ABP"}
            )
            segments.append(seg)
            spans.append((w_start, w_end))

    return segments, spans

In [ ]:
# devide train/val/test
@dataclass
class SegmentInfo:
    set_name: str
    origin_file: Path
    out_path: Path
    rows: int
    start_time: float
    end_time: float

def distribute_and_save(all_segments_per_file, out_root: Path, seed: int = RANDOM_SEED) -> pd.DataFrame:
    """
    random split into train/val/test and record a list
    """
    rng = random.Random(seed)
    sets = ["train", "val", "test"]
    manifest_rows = []

    for origin, segs, spans in all_segments_per_file:
        if not segs:
            continue

        idxs = list(range(len(segs)))
        rng.shuffle(idxs) 

        for i, seg_idx in enumerate(idxs):
            set_name = sets[i % 3]
            seg = segs[seg_idx]
            st, et = spans[seg_idx]

            subdir = out_root / set_name / origin.stem
            subdir.mkdir(parents=True, exist_ok=True)
            out_path = subdir / f"{origin.stem}_seg{i:04d}.csv"
            seg.to_csv(out_path, index=False)

            manifest_rows.append({
                "set": set_name,
                "origin_file": str(origin),
                "out_csv": str(out_path),
                "rows": int(len(seg)),
                "start_time": float(st),
                "end_time": float(et)
            })

    man_df = pd.DataFrame(manifest_rows)
    out_root.mkdir(parents=True, exist_ok=True)
    if not man_df.empty:
        man_df.to_csv(out_root / "manifest.csv", index=False)
    return man_df

In [ ]:
import math
import numpy as np
import pandas as pd

def split_one_csv_iter(csv_path: Path, window_sec: float = WINDOW_SEC):
    """
    generate files
    yield: (seg_df, w_start, w_end)，列为[t_sec, ECG, PPG, ABP]
    """
    try:
        df = pd.read_csv(csv_path, engine="pyarrow")
    except Exception as e:
        print(f"[SKIP] read fail {csv_path.name}: {e}")
        return

    if df.empty:
        print(f"[SKIP] empty file {csv_path.name}")
        return

    time_col = _detect_time_col(df)
    ecg_col  = _find_first_col_containing(df, "ECG")
    ppg_col  = _find_first_col_containing(df, "PPG")
    abp_col  = _find_first_col_containing(df, "ABP")

    missing = [n for n, c in [("time", time_col), ("ECG", ecg_col), ("PPG", ppg_col), ("ABP", abp_col)] if c is None]
    if missing:
        print(f"[SKIP] {csv_path.name}: no column: {', '.join(missing)}")
        return

    for c in [time_col, ecg_col, ppg_col, abp_col]:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    df = df.dropna(subset=[time_col]).sort_values(time_col).reset_index(drop=True)

    t = df[time_col].to_numpy()
    runs = _continuous_runs(t)

    for s, e in runs:
        run = df.iloc[s:e].copy()
        if len(run) < 2:
            continue
        t_run = run[time_col].to_numpy()
        tol   = _tolerance_from_dt(t_run)

        t0, t_last = float(t_run[0]), float(t_run[-1])
        total_dur  = t_last - t0
        if total_dur < window_sec - tol:
            continue

        n_windows = int(math.floor((t_last - t0) / window_sec))
        for k in range(n_windows):
            w_start = t0 + k * window_sec
            w_end   = w_start + window_sec
            mask = (run[time_col] >= w_start) & (run[time_col] < w_end)
            seg  = run.loc[mask].copy()
            if seg.empty:
                continue

            cov = float(seg[time_col].iloc[-1] - seg[time_col].iloc[0])
            if cov < (window_sec - tol):
                continue

            if seg[[ecg_col, ppg_col, abp_col]].isna().any().any():
                continue

            seg = seg[[time_col, ecg_col, ppg_col, abp_col]].rename(
                columns={time_col: "t_sec", ecg_col: "ECG", ppg_col: "PPG", abp_col: "ABP"}
            )
            yield seg, w_start, w_end


In [ ]:
# multi threads
import os, csv, threading, random, hashlib
from concurrent.futures import ThreadPoolExecutor, as_completed
from pathlib import Path

MAX_WORKERS = 24   # total threads
sets = ["train", "val", "test"]

OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)
csv_files = sorted([p for p in INPUT_DIR.glob("**/*.csv") if p.is_file()])
if not csv_files:
    raise FileNotFoundError(f"在 {INPUT_DIR} 下没有找到 CSV 文件")

# 初始化 manifest：写表头（如不存在）
manifest_path = OUTPUT_ROOT / "manifest.csv"
if not manifest_path.exists():
    with open(manifest_path, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=["set", "origin_file", "out_csv", "rows", "start_time", "end_time"])
        writer.writeheader()

# 共享的manifest写入器与锁（同一文件句柄，线程间共享，写入时加锁）
manifest_lock = threading.Lock()
_manifest_file = open(manifest_path, "a", newline="", encoding="utf-8")
_manifest_writer = csv.DictWriter(_manifest_file, fieldnames=["set", "origin_file", "out_csv", "rows", "start_time", "end_time"])

# 全局统计（上锁更新）
total_written = 0
set_counts = {"train": 0, "val": 0, "test": 0}

def process_one_file(csv_path: Path):
    """
    线程任务：处理单个CSV，逐片段切分->分配->写盘->记录manifest。
    返回：(written_this_file, local_counts_dict)
    """
    for sn in ("train", "val", "test"):
        (OUTPUT_ROOT / sn / csv_path.stem).mkdir(parents=True, exist_ok=True)
        
    global total_written  # 这里声明使用全局的 total_written
    import pandas as pd
    import hashlib, random

    per_file_idx = 0
    written_this_file = 0
    local_counts = {"train": 0, "val": 0, "test": 0}

    # 为该文件创建确定性的随机序列（保证可复现）
    seed_int = int(hashlib.md5(str(csv_path).encode("utf-8")).hexdigest()[:8], 16) ^ RANDOM_SEED
    rng = random.Random(seed_int)
    order = [0, 1, 2]
    rng.shuffle(order)

    try:
        for seg, st, et in split_one_csv_iter(csv_path, window_sec=WINDOW_SEC):
            set_name = sets[order[per_file_idx % 3]]
            if per_file_idx % 3 == 2:
                rng.shuffle(order)

            subdir = OUTPUT_ROOT / set_name / csv_path.stem
            #subdir.mkdir(parents=True, exist_ok=True)
            out_path = subdir / f"{csv_path.stem}_seg{per_file_idx:04d}.csv"

            seg.to_csv(out_path, index=False)

            # ★ 用锁保护：写 manifest + 更新计数
            with manifest_lock:
                _manifest_writer.writerow({
                    "set": set_name,
                    "origin_file": str(csv_path),
                    "out_csv": str(out_path),
                    "rows": int(len(seg)),
                    "start_time": float(st),
                    "end_time": float(et),
                })
                #_manifest_file.flush()

                total_written += 1            # 这里已是全局变量
                set_counts[set_name] += 1     # 字典项就地修改，无需 global

            per_file_idx += 1
            written_this_file += 1
            local_counts[set_name] += 1

        print(f"{csv_path.name}: 写出有效 6s 片段 {written_this_file} 个")

    except Exception as e:
        print(f"[ERROR] 处理 {csv_path.name} 出错: {e}")

    with manifest_lock:
        _manifest_file.flush()

    return written_this_file, local_counts


# 在线程池中并行处理多个CSV
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
    futures = [ex.submit(process_one_file, p) for p in csv_files]
    # 等待全部完成（异常会在 result() 时抛出）
    for fut in as_completed(futures):
        _ = fut.result()

# 关闭manifest文件
_manifest_file.close()

# 汇总
if total_written == 0:
    print("没有产出有效片段：请检查列名(含ECG/PPG/ABP)、时间列，以及数据是否存在空值。")
else:
    print(f"\n完成：共写出 {total_written} 个 6s 片段，线程数={MAX_WORKERS}")
    for k in ["train", "val", "test"]:
        print(f"  {k:>5}: {set_counts.get(k, 0)}")
    print(f"\n清单文件：{manifest_path}")
